In [ ]:
t11=0.4;t12=0.6;t13=0.9;o11=0.5;o12=0.3;o13=0.4

In [1]:
import torch
import torchvision.models as model
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, datasets
import os
import glob
from tqdm import tqdm
from PIL import Image
import random
from sklearn.model_selection import train_test_split
import numpy as np
import timm.models as tmodels

In [2]:
random_seed = 1120
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
import matplotlib.image as img
import matplotlib.pyplot as plt

def imshow(x):
    x = img.imread(x)
    plt.imshow(x)
    plt.show()

In [36]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# path1 = '/media/data2/rjsdn/zoom/data/dataset/train'
# path1 ='/media/data1/dataset/PGGAN/256x256/train'
# train_path='/media/data1/dataset/DFDC/train/'
a='/media/data2/rjsdn/zoom/data/dataset/test3/'
b='/media/data1/dataset/PGGAN/256x256/test/'
c='/media/data1/dataset/DFDC/test/'
train_path = '/media/data1/dataset/PGGAN/256xps 256/train/'
val_path = '/media/data2/rjsdn/zoom/data/eunju/test/'
test_path = '/media/data2/rjsdn/zoom/dataset/test1/'
# path2= '/media/data1/dataset/PGGAN/256x256/test'
#test = [ y for x in os.walk(path) for y in glob.glob(os.path.join(x[0], '*.jpg')) ]

SAVE_PATH='weights/R18P_D//'
os.makedirs(SAVE_PATH,exist_ok=True)

w,h=178,218

transform = transforms.Compose([
    transforms.Resize((w,h)),
    transforms.ToTensor()
])

train_img = datasets.ImageFolder(train_path,transform=transform)
val_img = datasets.ImageFolder(val_path,transform=transform)
test_img = datasets.ImageFolder(test_path,transform=transform)


b_size = 64

train_loader = DataLoader(train_img,batch_size=b_size,shuffle=True,num_workers=8)
val_loader = DataLoader(val_img,batch_size=b_size,shuffle=True,num_workers=8)
test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)







In [8]:
len(test_img)

4530

In [5]:
Model=model.resnet18(pretrained=False)
# Model=tmodels.xception(pretrained=False)
num_ftrs = Model.fc.in_features
Model.fc = nn.Linear(num_ftrs,2)
    
device = torch.device("cuda:0")
Model=Model.to(device)

opt = torch.optim.Adam(Model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [6]:
EPOCH=30

In [7]:
history={
    'train_acc':[],
    'val_acc':[],
    'train_loss':[],
    'val_loss':[],
}

best_loss = np.inf
best_acc = 0

for epo in range(EPOCH):
    train_loss=0
    train_acc=0
    Model.train()
    count=0
    for x, y in tqdm(train_loader):
        opt.zero_grad()
        x=x.to(device)
        y=y.to(device)
        output = Model(x)
        loss = criterion(output,y)
        _, index = torch.max(output,1)
        train_acc += (index==y).sum().float().item() / len(output)
        train_loss += float(loss.item())
        loss.backward(retain_graph=True)
        opt.step()
        count+=1
    
    train_loss /= count
    train_acc /= count

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    
    val_loss=0
    val_acc=0    
    Model.eval()
    with torch.no_grad():
        count=0
        for x,y in tqdm(val_loader):
            x=x.cuda()
            y=y.cuda()
            output=Model(x)
            loss = criterion(output,y)
            _, index = torch.max(output,1)
            val_acc += (index==y).sum().float().item() / len(output)
            val_loss += float(loss.item())
            count+=1
        val_acc/=count
        val_loss/=count
    
    print(f'[{epo+1}/{EPOCH}] TL={train_loss: .8f} TA={train_acc*100:.3f} || VL={val_loss:.8f} VA={val_acc*100:.2f}')
    
    
    if best_loss > val_loss:
        print('best weights changed!!!')
        best_loss=val_loss
        best_acc=val_acc
        torch.save(Model.state_dict(),SAVE_PATH+'/best.pth')
        
    
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    
    
print("\n\nTrain Done")

100%|██████████| 183/183 [00:27<00:00,  6.67it/s]


[1/30] TL= 0.50997029 TA=73.350 || VL=0.68284809 VA=67.98
best weights changed!!!


100%|██████████| 183/183 [00:24<00:00,  7.40it/s]


[2/30] TL= 0.19373093 TA=91.997 || VL=0.25086895 VA=90.41
best weights changed!!!


100%|██████████| 183/183 [00:25<00:00,  7.09it/s]


...

[28/30] TL= 0.00743105 TA=99.731 || VL=0.02567415 VA=99.28


100%|██████████| 183/183 [00:25<00:00,  7.25it/s]


[29/30] TL= 0.00481292 TA=99.851 || VL=0.02435313 VA=99.21


100%|██████████| 183/183 [00:24<00:00,  7.48it/s]

[30/30] TL= 0.00566722 TA=99.806 || VL=0.02918706 VA=99.16


Train Done


In [37]:
# testmodel=tmodels.xception(pretrained=False)
testmodel=tmodels.resnet18(pretrained=False)
num_ftrs = testmodel.fc.in_features
testmodel.fc = nn.Linear(num_ftrs,2)
testmodel.load_state_dict(torch.load(SAVE_PATH+'/best.pth'))
testmodel.cuda()
# print(f'best acc={best_acc*100:.2f}')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [38]:
from sklearn.metrics import classification_report

test_loss=0
acc = 0
count=0
p,r=torch.tensor([0]),torch.tensor([0])
testmodel.eval()
with torch.no_grad():
    for x,y in tqdm(test_loader):
        count+=1
        x=x.cuda()
        y=y.cuda()
        output = testmodel(x)
        pred=output.argmax(dim=1)
        pred = pred ^ 1
        p=torch.cat([p,pred.to("cpu")])
        r=torch.cat([r,y.to("cpu")])
        acc += pred.eq(y.view_as(pred)).sum().item()
    acc /= len(test_img)
    print("ACC : %.5f"%(acc*100))

100%|██████████| 250/250 [00:04<00:00, 51.77it/s]

ACC : 50.60000


In [ ]:
nn.

In [ ]:
#### import pandas as pd

pd.DataFrame(history).to_csv(SAVE_PATH+'/res.csv', index=False)

In [39]:
CM = confusion_matrix(r,p)
print(CM)
print(f1_score(r,p))
print(recall_score(r,p))
print(precision_score(r,p))

[[1875  126]
 [1850  150]]
0.13181019332161686
0.075
0.5434782608695652


f1,recall,precision

X DFDC
68.375
0.6806362029790457
0.674
0.6874043855175931

X PGGAN
54.125
0.27384250098931534
0.173
0.6565464895635673

R18 DFDC
69.375
0.6257256339749466
0.512
0.8043990573448547

R18 PGGAN
50.6
0.13181019332161686
0.075
0.5434782608695652